In [1]:
import numpy as np
import pandas as pd
import pymongo as mongo
import re


import nltk
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, PCA, TruncatedSVD, LatentDirichletAllocation
from sklearn.utils.extmath import randomized_svd

from textblob import TextBlob, Word, WordList
from gensim import models, corpora, similarities, matutils

from sys import getsizeof
import psutil
import pickle
from multiprocessing import Pool

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [2]:
psutil.virtual_memory()

svmem(total=8589934592L, available=3508666368L, percent=59.2, used=8142651392L, free=444854272L, active=3778441216L, inactive=3063812096L, wired=1300398080L)

In [3]:
pd.options.display.max_columns = 50

In [4]:
client = mongo.MongoClient()

In [5]:
client.database_names()

[u'local', u'mydb', u'project4']

In [6]:
db = client.get_database('project4')

In [7]:
journals = db.collection_names()[:-1] # drop system.indexes...

In [8]:
# Fixing poor naming schema from before
for j in journals:
    coll = db.get_collection(j)
    coll.update_many({'Max_Topic_Name':'Attention??'}, {"$set":{'Max_Topic_Name':"Attention"}})
    coll.update_many({},{"$rename": {"Attention??":"Attention"}})

CountVectorizer()
#### input can equal a list of filenames to read (or file objects)

#### analyzer:
    - default is "word"
    - Extracts each interpreted "word" as features
    
#### The Analyzer is composed of 2 steps:

1) Preprocessing
    - This is the general string formatting that is done before tokenizing
    - lowercasing, removing accents
    - does not seem to include stemming/lemmatizing :-/??
    - though you may be able to pass in a function that does whatever you want...
2) Tokenizing
    - Creating a vocabulary based on what is considered to be a "word"
    - by default, all groups of 2 or more consecutive alpha-numeric characters are considered "words".
        * punctuation is ignored and always splits "words"
    - you can set the definition using token_pattern = regex...


#### ngram_range:
    - number of ngrams to consider for features
    - format is (min, max)
#### stop_words:
    - can be 'english' or list of stopwords
#### lowercase:
    - Boolean default is True
    
    
#### min_df and max_df:
    - set min and max occurences required for each ftr


In [9]:
def JournalData(j):
    cvec = CountVectorizer(stop_words='english')
    collec = db.get_collection(j)    
    
    # find returns a curser which gets a list of dictionaries
    data = collec.find({})
    # in addition to our features include columns for Journal and Date
    ids = []
    journals = []
    dates = [] 
    text_data = []
    for i in range(data.count()):
        doc = data.next()
        ids.append(doc['_id'])
        dates.append(doc['date'])
        text_data.append(doc['text'])
        journals.append(j)
    return ids, journals, dates, text_data



In [10]:
ids = []
jnls = []
dates = []
text_data = []

In [11]:
# BUILD LISTS CONTAINING ALL JOURNAL_NAMES DATES AND ARTICLES

for j in journals:
#     if j in jnls:
#         continue
    i, j, d, t = JournalData(j)
    ids.extend(i)
    jnls.extend(j)
    dates.extend(d)
    text_data.extend(t)
    print(getsizeof(jnls))
    if getsizeof(jnls) > 1000000000:
        print('SHIT THATS BIG (THATS WHAT SHE SAID)')
        break

384
920
920
920
920
3936
3936
4488
4488
4488
5896
5896
5896
5896
9088
15152
15152
15152
15152
15152
18368
18368
18368
18368
18368
18368
18368
25912
29312
29312
29312
29312
29312
29312
48432
48432
48432
48432
48432
48432
48432
48432
48432
48432
48432
48432
48432
48432
48432
48432
48432
48432
48432
48432
48432
48432
48432
55296
55296
55296
55296
55296
55296
55296
55296
55296
55296
55296
55296
55296
55296
55296
55296
55296
55296
55296
55296
62768
62768
62768
62768
62768
62768
62768
62768
62768
62768
62768
62768
62768
62768
62768
62768
70912
70912
70912
70912
70912
70912
70912
70912
70912
70912
70912
70912
70912
70912
70912
70912
80584
80584
80584
80584
80584
80584
80584
80584
80584
80584
80584
80584
91128
91128
91128
91128
91128
91128
91128
91128
91128
109712
109712
109712
109712
109712
109712
109712
109712
125480
125480
125480
125480
125480
125480
125480
125480
125480
125480
125480
125480
125480
125480
125480
125480
125480
125480
125480
125480
125480
141544
141544
141544
141544
141544
14

In [12]:
print(len(ids), len(jnls), len(dates), len(text_data))

(25211, 25211, 25211, 25211)


## CLEANING TEXT DATA

In [13]:
body = re.compile(r'== ?[bB]ody ')

def clean_doc(doc):
    # needs textblob
    
    # GET RID OF NEWLINES
    doc = doc.replace('\n', ' ')
    
    # STRIP HEADING INFO
    # if there's no body section, get rid of it
    if not re.search(body, doc):
        return None
    mtch = re.search(body,doc)
    # strip anything before match
    doc = doc[mtch.start():]

    # LOWERCASE!
    doc = doc.lower()

    #SINGULARIZE
    # this seriously slows it down!
    #doc = ' '.join([Word(w).singularize() for w in doc.split()])
    return doc
        
    
    



In [14]:
idx = 0
cleaned = []

In [15]:
pool = Pool(4)
cleaned = pool.map(clean_doc, text_data)
pool.close()
len(cleaned)

25211

In [16]:
psutil.virtual_memory()

svmem(total=8589934592L, available=1082322944L, percent=87.4, used=3512606720L, free=16064512L, active=1071554560L, inactive=1066258432L, wired=1374793728L)

In [17]:
# USE POOL - MUCH FASTER!
# Obviously this takes a while...
# Maybe too long
# len(text_data) # 25370


# for t in iter(text_data[idx:]):
#     print(idx)
#     cleaned.append(clean_doc(t))
#     idx+=1
# len(cleaned)

In [18]:
# REMOVE STOP WORDS

# stop_words = nltk.corpus.stopwords.words()
# len(stop_words) # 2781 ... i think has other languages need only english

# englishStopWords = None
# with open('csvs/EngStopWords.csv', 'r') as f:
#     englishStopWords = f.read().split(', ')
    
    
# len(englishStopWords) # 174 better


In [20]:
with open('WranglingPickles/cleaned.pkl', 'rb') as f:
    cleaned = pickle.load(f)

In [21]:
len(cleaned)

25211

In [22]:
# Join With Jrnl Name and Date

Joined = list(zip(ids,jnls, dates, range(len(cleaned))))



# REMOVE NULLS
Joined = [a for a in Joined if cleaned[a[3]]]
cleaned = list(filter(None, cleaned))

## COUNTVECTORIZE

In [23]:
len(cleaned)

25168

In [25]:
print(len(cleaned)) # 25327



cvectr = CountVectorizer(stop_words='english', min_df=10, max_df = 0.5)
txt_ftrs = cvectr.fit_transform(cleaned)


txt_ftrs

25168


<25168x126204 sparse matrix of type '<type 'numpy.int64'>'
	with 26137222 stored elements in Compressed Sparse Row format>

In [26]:
corpus = matutils.Sparse2Corpus(txt_ftrs)

In [27]:
wordDic = {d[1]: d[0] for d in cvectr.vocabulary_.items()}

print(len(wordDic))
print(max(wordDic.keys()))

126204
126203


## THINGS TO TRY:

#### Summarize
    - Apply PCA, SVD, NMF, LDA
    - Interpret results... aka grouping?? how?
    

#### Features:
    - POS
    - Somehow incorporate Parse Trees?
    
    
#### Beyond:
    - Word2vec
    - Heirarchical Clustering

#### PCA

In [28]:
# PCA Doesn't work cuz it doesn't accept sparse

#### LDA

In [29]:
# MEMORY ERROR

#### SVD

In [30]:
# GOING WITH NMF BECAUSE SVD TOPICS ARE CRUMMY

#### WHAT I SEE:
    - Remove Plurals
    - Adjust for Tenses
    - Can we fuse numbers/units for measurements?
    - Eliminate Random Numbers

#### NMF

In [43]:
nmf = NMF(init = 'nndsvd', n_components=15)

nmf.fit(txt_ftrs)

NMF(alpha=0.0, beta=1, eta=0.1, init='nndsvd', l1_ratio=0.0, max_iter=200,
  n_components=15, nls_max_iter=2000, random_state=None, shuffle=False,
  solver='cd', sparseness=None, tol=0.0001, verbose=0)

In [44]:
nmf.components_.shape

(15, 126204)

In [45]:
# get top 25 significant words for each component
top25 = {}

for i in range(len(nmf.components_)):# for each component
    # sort the words in the components by importance, but grab the original indices
    idxs = np.argsort(nmf.components_[i])[-1:-26:-1]
    # use idxs to extract corresponding words from wordDic
    top25['ftr_'+str(i)] = [wordDic[x] for x in idxs]

In [46]:
top25

{'ftr_0': [u'usepackage',
  u'document',
  u'end',
  u'begin',
  u'minimal',
  u'documentclass',
  u'amsbsy',
  u'mathrsfs',
  u'amsmath',
  u'amssymb',
  u'amsfonts',
  u'oddsidemargin',
  u'upgreek',
  u'wasysym',
  u'12pt',
  u'69pt',
  u'setlength',
  u'spike',
  u'fig',
  u'frac',
  u'stimulus',
  u'sf',
  u'mathrm',
  u'eq',
  u'hat'],
 'ftr_1': [u'protein',
  u'expression',
  u'gene',
  u'cell',
  u'genes',
  u'proteins',
  u'receptor',
  u'binding',
  u'signaling',
  u'biol',
  u'neuronal',
  u'1016',
  u'kinase',
  u'regulation',
  u'neurosci',
  u'factor',
  u'dependent',
  u'dna',
  u'mol',
  u'mutations',
  u'molecular',
  u'transcription',
  u'mouse',
  u'complex',
  u'synaptic'],
 'ftr_10': [u'pain',
  u'spinal',
  u'nerve',
  u'injury',
  u'cord',
  u'surgery',
  u'motor',
  u'stimulation',
  u'tumor',
  u'months',
  u'sclerosis',
  u'mri',
  u'surgical',
  u'syndrome',
  u'epilepsy',
  u'chronic',
  u'muscle',
  u'ms',
  u'therapy',
  u'lesions',
  u'fig',
  u'symptoms'

In [47]:
text_nmf = pd.DataFrame(nmf.transform(txt_ftrs), columns = ['ftr_' + str(i) for i in range(len(top25.keys()))])
getsizeof(text_nmf) # 3mb
text_nmf.head(10)


,ftr_0,ftr_1,ftr_2,ftr_3,ftr_4,ftr_5,ftr_6,ftr_7,ftr_8,ftr_9,ftr_10,ftr_11,ftr_12,ftr_13,ftr_14
0,0.001273,0.000000,0.247913,0.000000,0.199396,0.471529,0.167966,0.060914,0.017921,2.139161,0.000000,0.000000,0.961869,0.186803,0.538371
1,0.004978,0.000000,0.000000,0.317037,0.136185,0.468468,0.206836,0.000000,0.084690,0.496419,0.512749,0.000000,0.000000,0.064152,0.000000
2,0.033887,0.043568,0.000000,0.722161,0.448714,0.774374,0.223858,0.000000,0.000000,0.000000,0.502211,0.000000,0.000000,0.027228,0.000000
3,0.000000,0.000000,0.000000,1.271206,0.000000,0.000000,0.000000,0.010501,0.000000,0.000000,0.000000,0.006899,0.062925,0.821910,0.798151
4,0.000000,0.000000,0.024419,0.000000,0.153504,2.570706,0.249871,0.000000,0.000000,0.083720,0.515074,0.000000,0.013573,0.000000,0.000000
5,0.045722,0.000439,0.000000,0.645580,0.938122,0.774892,0.098891,0.052211,0.002105,0.000000,0.330594,0.049817,0.000000,0.179257,0.000000
6,0.004432,0.346745,0.000000,0.000000,1.069199,1.313052,0.289753,0.000000,0.000000,0.182861,0.088905,0.003165,0.000000,0.000000,0.000000
7,0.010086,0.071281,0.000000,0.000000,0.559325,0.069781,0.134578,0.259437,0.000000,0.000000,0.240100,0.134084,0.000887,1.184734,0.000000
8,0.006453,0.035886,0.000000,0.044988,0.123514,0.049368,0.566547,0.000000,0.000000,0.017823,0.742868,0.000000,0.000000,0.028659,0.000000
9,0.003894,0.002947,0.067765,0.595825,0.019437,0.007981,0.000000,0.048926,0.099766,0.000000,0.067690,0.032262,0.023007,0.039945,0.244947


In [48]:
components_df = pd.DataFrame(top25)

In [49]:
getsizeof(components_df) # 23kb
components_df

,ftr_0,ftr_1,ftr_10,ftr_11,ftr_12,ftr_13,ftr_14,ftr_2,ftr_3,ftr_4,ftr_5,ftr_6,ftr_7,ftr_8,ftr_9
0,usepackage,protein,pain,stroke,children,cortex,memory,depression,visual,cells,neurons,mice,ad,sleep,rats
1,document,expression,spinal,cerebral,social,connectivity,cognitive,schizophrenia,stimulus,cell,neuron,fig,alzheimer,rem,receptor
2,end,gene,nerve,risk,autism,regions,task,disorder,task,fig,synaptic,mouse,amyloid,insomnia,receptors
3,begin,cell,injury,ischemic,disorder,cortical,learning,psychiatry,trials,expression,spike,wt,dementia,eeg,stress
4,minimal,genes,cord,acute,adhd,network,performance,symptoms,stimuli,stem,firing,expression,cognitive,circadian,induced
5,documentclass,proteins,surgery,blood,disorders,temporal,training,disorders,ms,anti,neuronal,animals,aβ,syndrome,dopamine
6,amsbsy,receptor,motor,artery,asd,frontal,hippocampal,placebo,responses,μm,input,transgenic,tau,disorders,rat
7,mathrsfs,binding,stimulation,outcome,child,matter,hippocampus,bipolar,neural,adult,cortical,wild,impairment,wake,cocaine
8,amsmath,signaling,tumor,hemorrhage,anxiety,gyrus,working,depressive,target,growth,cortex,supplementary,beta,night,1016
9,amssymb,biol,months,flow,behavior,areas,tasks,mg,condition,pituitary,neurosci,neurons,csf,deprivation,drug


In [51]:
# text_data, cleaned, cvectr, nmf, text_nmf, components_df
with open('WranglingPickles/text_data.pkl','wb') as f:
    pickle.dump(text_data, f, 2)
    
with open('WranglingPickles/cleaned.pkl','wb') as f:
    pickle.dump(cleaned, f, 2)
    
with open('WranglingPickles/cvectr.pkl','wb') as f:
    pickle.dump(cvectr, f, 2)
    
with open('WranglingPickles/nmf.pkl','wb') as f:
    pickle.dump(nmf, f, 2)
    
with open('WranglingPickles/text_nmf.pkl','wb') as f:
    pickle.dump(text_nmf, f, 2)
    
with open('WranglingPickles/components_df.pkl','wb') as f:
    pickle.dump(components_df, f, 2)
    
with open('WranglingPickles/Joined.pkl','wb') as f:
    pickle.dump(Joined, f, 2)